In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, IBMQ, QuantumRegister, ClassicalRegister, execute, BasicAer
from qiskit.compiler import assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import CircuitComposer
from qiskit.providers.aer import QasmSimulator, AerSimulator
import numpy as np
from numpy import pi

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [ ]:
# Input
def load_input(circ,n,PLAIN_TEXT,operand2):
    for i in range(n):
        if PLAIN_TEXT[i]=='1':
            circ.x(operand2[len(PLAIN_TEXT)-1-i])

In [ ]:
# key
def load_key(circ,n,operand1):
    for i in range(n):
        circ.h(operand1[n-1-i])

In [ ]:
# cipher
def cipher(circ,adder,anc,operand1,operand2,cr):
    circ.append(adder, [anc[0]] + operand1[:] + operand2[:] + [anc[1]])
    circ.measure(operand2[:] + [anc[1]], cr) #changes will be on operand 2 in other word (here) plain_text is changed 

In [ ]:
# def my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n):
    #Compute cipher text possibilities by XORing
    
def my_oracle(circ,adder,operand1,operand2,anc,cr,n):    
    # cipher(qc, r_key, r_text, n)
    cipher(circ,adder,anc,operand1,operand2,cr)
    
    #Checking whether the generated cipher text is equal to the given cipher text
    for i in range(n):
        circ.cx(r_cipher[i], r_ancilla[i])
        circ.cx(r_text[i], r_ancilla[i])
        circ.x(r_ancilla[i])
        
    #Set 'output' bit if the cipher text is matched
    qc.mcx(r_ancilla, r_output)
    
    qc.barrier()
    
    #Uncompute cipher to reset ancilla & plain text qubits
    #Reset ancilla qubit
    for i in range(n):
        qc.x(r_ancilla[i])
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])
    qc.barrier()
    #Reset plain text by inverse cipher process
    cipher(qc, r_key, r_text, n)
    
    qc.barrier()

In [ ]:
#The diffuser function is available in Qiskit
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    #Apply transformation |s> -> |00...0> (H gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    #Apply transformation |00...0> -> |11...1> (X gates)
    for qubit in range(nqubits):
        qc.x(qubit)    
    #Do multi-controlled Z gate
    qc.h(nqubits - 1)
    qc.mct(list(range(nqubits - 1)), nqubits - 1) #multi-controlled Toffoli
    qc.h(nqubits - 1)
    #Apply transformation |11...1> -> |00...0> (X gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    #Apply transformation |00...0> -> |s> (H gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    #We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s
    

In [ ]:
# converting plain text to binary
plain_text = "a"
# printing original string 
print("The original string is : " + str(plain_text))
 
# ENCRYPTED_TEXT=''

# KEY="101" #5 in decimal
# n_k=len(KEY)
# print("key=",int(KEY, 2))

In [ ]:
from qiskit.circuit.library import CDKMRippleCarryAdder 
res = ''.join(format(ord(plain_text), '08b')) # converting to binary
PLAIN_TEXT=str(res) 
# print(PLAIN_TEXT)
n_p=len(PLAIN_TEXT)

adder = CDKMRippleCarryAdder(n_p, kind='full', name='Full Adder') # kind='full' indicates full adder
operand1 = QuantumRegister(n_p, name='key')
operand2 = QuantumRegister(n_p, name='plain_text')
cipher_text= QuantumRegister(n_p, name='cipher_text')
anc = QuantumRegister(2, name='a') # to store carry in and carry out
cr = ClassicalRegister(n_p+1,name='cr')

circ = QuantumCircuit(operand1, operand2, cipher_text, anc, cr)

In [ ]:
#loading plain text
plain_text = '0101'
load_input(circ,n_p,PLAIN_TEXT,operand2)

In [ ]:
#Preparing key in a superposition state
load_key(circ,n_p,operand1)

In [ ]:
#loading known cipher text
cipher_text = '1001'
load_input(qc, r_cipher, cipher_text, n)

In [ ]:
n = 4
r_text = QuantumRegister(n, 't')
r_key = QuantumRegister(n, 'k')
r_output = QuantumRegister(1, name = 'o')
r_cipher = QuantumRegister(n, 'c')
r_ancilla = QuantumRegister(n, 'a')

r_class = ClassicalRegister(n)

qc = QuantumCircuit(r_text, r_key, r_cipher, r_ancilla, r_output, r_class)
#loading plain text
plain_text = '0101'
load_input(qc, r_text, plain_text, n)

#Preparing key in a superposition state
load_key(qc, r_key, n)

#loading known cipher text
cipher_text = '1001'
load_input(qc, r_cipher, cipher_text, n)

#Preparing output qubit
qc.x(r_output)
qc.h(r_output)
qc.barrier()

#First Iteration
my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
qc.barrier()
qc.append(diffuser(n), r_key)
qc.barrier()

#Second Iteration
my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
qc.barrier()
qc.append(diffuser(n), r_key)
qc.barrier()

qc.measure(r_key, r_class)
qc.draw()
